In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
gen_train=ImageDataGenerator(rescale=1/255,shear_range=0.2,zoom_range=0.2,brightness_range=(0.1,0.5),horizontal_flip=True)


In [3]:
train_data=gen_train.flow_from_directory('C:/Users/Dhruv Sanghvi/Desktop/Waste Segmentation/recycling waste')

Found 8369 images belonging to 9 classes.


In [4]:
from tensorflow.keras.applications.vgg16 import VGG16
vgg16=VGG16(input_shape=(224,224,3),weights='imagenet',include_top=False)

In [5]:
for layer in vgg16.layers:
    layer.trainable=False

In [6]:
from tensorflow.keras import layers

In [7]:
x = layers.GlobalAveragePooling2D()(vgg16.output)

In [8]:
prediction=layers.Dense(units=9,activation='softmax')(x)

In [9]:
model=tf.keras.models.Model(inputs=vgg16.input,outputs=prediction)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [ ]:
model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=['accuracy'])
result=model.fit(train_data,epochs=28,steps_per_epoch=len(train_data))

Epoch 1/28
 87/262 [========>.....................] - ETA: 14:48 - loss: 2.1430 - accuracy: 0.1875

In [ ]:
from tensorflow.keras.preprocessing import image
output_class=["batteries","clothes","e-waste","glass","light bulbs","metal","organic","paper","plastic"]
def waste_prediction(new_image):
    test_image=image.load_img(new_image,target_size=(224,224))
    plt.imshow(test_image)
    plt.show()
    
    test_image=image.img_to_array(test_image)/255
    test_image=np.expand_dims(test_image,axis=0)
    predicted_array=model.predict(test_image)
    predicted_value=output_class[np.argmax(predicted_array)]
    predicted_accuracy=round(np.max(predicted_array)*100,2)
    print("Your Waste Material is ",predicted_value,"with", predicted_accuracy,"accuracy")

In [ ]:
plt.title("Accuracy")
plt.plot(result.history["accuracy"])
plt.show()

In [ ]:
plt.title("Loss")
plt.plot(result.history["loss"])
plt.show()

In [ ]:
model.save("ClassifyWaste.h5")